## Import all Libraries

In [1]:
# %matplotlib inline
import array
import numpy as np                                        
import pandas as pd
from pathlib import Path
import openpyxl
import os
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from scipy.ndimage.filters import gaussian_filter
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.notebook_repr_html', True)

In [2]:
file_path_bin = "D:/MASS/14 inch IPIG ILI run data/IOCL_DELHI_PANIPAT_RUN_5_DEC_2015/Data"
file_path_cd = "D:/MASS/14 inch IPIG ILI run data/IOCL_DELHI_PANIPAT_RUN_5_DEC_2015/Individual CD"

In [3]:
feature = ''
individual_cd = {}
pathlist = Path(file_path_cd).glob('**/*.xlsx')
for pathObj in pathlist:
    path_in_str = str(pathObj)
    print(path_in_str.split('\\')[-1])
    cd_data = pd.read_excel(path_in_str, sheet_name='Sheet3')
    cd_data.columns = np.arange(0,len(cd_data.columns))
    cd_data = cd_data[[1,3,4,11,5]].dropna(thresh = 1)
    cd_data[11] = cd_data[11].str.lower()
    cd_data = cd_data[cd_data[11].str.contains(feature)]
    cd_data.columns = ['Page_Number', '5k_Sample_Count', 'Abs_Val', 'Feature', 'Central_Sensor']
    key = int(path_in_str.split('\\')[-1].split('_')[1])
    
    individual_cd['CD_' + str(key)] = []
    individual_cd['CD_' + str(key)].append(cd_data)
    
#Appending names of associate bin file at index 1 of the lists 
pathlist = Path(file_path_bin).glob('**/*.bin')
for pathObj in pathlist:
    path_in_str = str(pathObj)
    key  = 'CD_' + path_in_str.split('\\')[-1].split('_')[1]
    for value in individual_cd:
        if (value == key):
            individual_cd[value].append(path_in_str.split('\\')[-1])

CD_01_Manoj.xlsx
CD_02_Bharti.xlsx
CD_03_Rakesh.xlsx
CD_04_Dilpreet.xlsx
CD_05_Deepti.xlsx
CD_06_Deepa.xlsx
CD_07_Dilpreet_Manoj.xlsx
CD_08_Ashwani_Bharti.xlsx
CD_09_Deepti_Deepa.xlsx
CD_10_Manoj.xlsx


# Reading Bin files, processing and Storing 

In [6]:
pig_run_name = file_path_bin.split("/")[-2]
sigma_for_filtering = 2
distance_count = 5000
primary_sensors_count = 84                                
secondary_sensors_count = 72                              
other_sensors_count = 8
pri_skip = 1
sec_skip = 1
other_skip = 1
last_primary_channel = primary_sensors_count*pri_skip
last_secondary_channel = last_primary_channel + secondary_sensors_count*sec_skip
last_other_channel = last_secondary_channel  + other_sensors_count*other_skip
max_channels = last_other_channel
primary_data = np.zeros((primary_sensors_count, distance_count), dtype = np.float)
secondary_data = np.zeros((secondary_sensors_count, distance_count), dtype = np.float)
other_data = np.zeros((other_sensors_count, distance_count), dtype = np.float)
buffer = np.zeros((max_channels, distance_count), dtype = np.float)


temp_list = pd.DataFrame()
labelled_data = pd.DataFrame()
dict_cd_data = {}
dict_run_data = {}
for k in individual_cd:
    file_path = file_path_bin + '/' + individual_cd[k][1]
    print(k, file_path)
    sid = 25
    lister = []
    fid = open(file_path, 'rb')
    fid.seek(0,0)
    EOF = fid.seek(0,2)
    print(EOF)
    total_page_count = int(EOF/(5000*max_channels))
    print(total_page_count)
    num = 0
    while (num < total_page_count):
        fid.seek(num*(primary_sensors_count + secondary_sensors_count + other_sensors_count)*distance_count,0)
        arr = array.array("b")
        arr.fromfile(fid, ((max_channels)*distance_count))
        var1 = prepare_data(arr)
        lister.append(var1)
        num = num + 1
        if(num%100==0): print(num, 'page processed')
    fid.close() 
    dict_cd_data[k] = lister
dict_run_data[pig_run_name] = dict_cd_data
%store dict_run_data

CD_1 D:/MASS/14 inch IPIG ILI run data/IOCL_DELHI_PANIPAT_RUN_5_DEC_2015/Data/DATA_1_18.12.2015_18.10.17.bin
700034000
853
100 page processed
200 page processed
300 page processed
400 page processed
500 page processed
600 page processed
700 page processed
800 page processed
CD_2 D:/MASS/14 inch IPIG ILI run data/IOCL_DELHI_PANIPAT_RUN_5_DEC_2015/Data/DATA_2_18.12.2015_20.32.34.bin
700025800
853
100 page processed
200 page processed
300 page processed
400 page processed
500 page processed
600 page processed
700 page processed
800 page processed
CD_3 D:/MASS/14 inch IPIG ILI run data/IOCL_DELHI_PANIPAT_RUN_5_DEC_2015/Data/DATA_3_18.12.2015_22.54.51.bin
700075000
853
100 page processed
200 page processed
300 page processed
400 page processed
500 page processed
600 page processed
700 page processed
800 page processed
CD_4 D:/MASS/14 inch IPIG ILI run data/IOCL_DELHI_PANIPAT_RUN_5_DEC_2015/Data/DATA_4_19.12.2015_01.17.10.bin
700034000
853
100 page processed
200 page processed
300 page proce

MemoryError: 

In [7]:
len(dict_run_data)

0

In [5]:
def prepare_data(arr):
    k =  0
    for iota in range(0, distance_count):
        for j in range(0, max_channels):
            buffer[j,iota]=arr[k]
            k=k+1
    
    p1 = 0     
    for channel in range(0, last_primary_channel):
        for samp in range(0, distance_count):
            temp = buffer[channel,samp]
            temp2 = 511
            primary_data[p1, samp] = ((temp*4) + temp2)*5/2048
        p1 = p1 + 1
        
#     p2 = 0     
#     for channel in range(last_primary_channel, last_secondary_channel):
#         for samp in range(0, distance_count):
#             temp = buffer[channel,samp]
#             temp2 = 511
#             secondary_data[p2, samp] = ((temp*4) + temp2)*5/2048
#         p2 = p2 + 1
        
    p3 = 0     
#     for channel in range(last_secondary_channel, last_other_channel):
#         for samp in range(0, distance_count):
#             temp = buffer[channel,samp]
# #             temp2 = 511
#             other_data[p3, samp] = temp 
# #     ((temp*4) + temp2)*5/2048
#         p3 = p3 + 1
        
    pri_parent = [primary_data]
#     sec_parent = [secondary_data]
#     other_parent = [other_data]

    for value in range(0, len(pri_parent)):
        pri_parent[value] = pd.DataFrame(pri_parent[value]).transpose().dropna()
#     for value in range(0, len(sec_parent)):
#         sec_parent[value] = pd.DataFrame(sec_parent[value]).transpose().dropna()
#     for value in range(0, len(other_parent)):
#         other_parent[value] = pd.DataFrame(other_parent[value]).transpose().dropna()

    var1 = pri_parent[0].copy()
#     var2 = sec_parent[0].copy()
#     var3 = other_parent[0].copy()
    #Plotting grayscale raw
#     plot_grayscale(var1.transpose(), abs_folder_path, 'primary/raw/image_'+ str(page_number))
#     plot_grayscale(var2.transpose(), abs_folder_path, 'secondary/raw/image_'+ str(page_number))
    
    scaler = MinMaxScaler()
    var1 = pd.DataFrame(scaler.fit_transform(var1))
    var1 = pd.DataFrame(gaussian_filter(var1, sigma=sigma_for_filtering))
    return var1 #, var2, var3